In [63]:
"""
Collaborative Filtering ALS Recommender System using Spark MLlib adapted from
the Spark Summit 2014 Recommender System training example.

Developed By: Pranav Masariya
Supervisor: Dr. Magdalini Eirinaki
"""

import os
import numpy as np
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pyspark.mllib.recommendation import ALS
from pyspark.ml.recommendation import ALS as mlals
from pyspark.ml.evaluation import RegressionEvaluator

import math
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator

In [64]:
# Calling spark session to register application
spark = SparkSession \
    .builder \
    .appName("Recom") \
    .config("spark.recom.demo", "1") \
    .getOrCreate()

In [65]:
"""
Loading and Parsing Dataset
    Each line in the ratings dataset (ratings-1.csv) is formatted as:
         userId,movieId,rating
    Each line in the movies (movies.csv) dataset is formatted as:
        movieId,title

""" 

# Load ratings
ratings_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("ratings-1.csv")

In [66]:
ratings_df

DataFrame[userId: int, movieId: int, rating: int]

In [67]:
"""
For the simplicity of this tutorial
    For each line in the ratings dataset, we create a tuple of (UserID, MovieID, Rating). 
"""

#ratings_df = ratings_df.drop('timestamp')
ratings_df.show(30)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   149|      2|     5|
|   149|      3|     3|
|   149|      5|     3|
|   149|      7|     2|
|   149|      9|     4|
|   149|     10|     4|
|   149|     11|     4|
|   149|     12|     4|
|   149|     13|     4|
|   149|     14|     3|
|   149|     16|     5|
|   149|     18|     4|
|   149|     20|     5|
|   149|     21|     4|
|   969|      5|     5|
|   969|     14|     3|
|   969|     15|     1|
|   969|     16|     1|
|   969|     21|     2|
|   589|      1|     4|
|   589|      2|     5|
|   589|      3|     5|
|   589|      4|     4|
|   589|      5|     3|
|   589|      6|     3|
|   589|      7|     2|
|   589|      8|     4|
|   589|      9|     5|
|   589|     10|     5|
|   589|     11|     3|
+------+-------+------+
only showing top 30 rows



In [68]:
# Load movies
movies_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("movies.csv")

In [69]:
movies_df.show(5)

+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|      1|Rogue One / Star ...|
|      2|          Fight Club|
|      3|   Lord of the Rings|
|      4|              Trolls|
|      5|       Despicable Me|
+-------+--------------------+
only showing top 5 rows



In [70]:
"""
In order to determine the best ALS parameters, we will use the small dataset. 
We need first to split it into train, validation, and test datasets.
"""
(trainingData,validationData,testData) = ratings_df.randomSplit([0.6,0.2,0.2])

In [71]:
# Prepare test and validation set. They should not have ratings

validation_for_predict = validationData.select('userId','movieId')
test_for_predict = testData.select('userId','movieId')

In [72]:
"""
Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by 
using Alternating Least Squares. The implementation in MLlib has the following parameters:

    1. numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
    2. rank is the number of latent factors in the model.
    3. iterations is the number of iterations to run.
    4. lambda specifies the regularization parameter in ALS.
    5. implicitPrefs specifies whether to use the explicit 
        feedback ALS variant or one adapted for implicit feedback data.
    6. alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline 
        confidence in preference observations.

"""

seed = 5 #Random seed for initial matrix factorization model. A value of None will use system time as the seed.
iterations = 10
regularization_parameter = 0.01 #run for different lambdas - e.g. 0.01
ranks = [4, 8, 12] #number of features
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

In [73]:
# Let us traing our dataset and check the best rank with lowest RMSE
# predictAll method of the ALS takes only RDD format and hence we need to convert our dataframe into RDD
# df.rdd will automatically converts Dataframe into RDD

for rank in ranks:
    model = ALS.train(trainingData, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validationData.rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) # RMSE Error
    errors[err] = error
    err += 1
    print ('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 1.75254044445
For rank 8 the RMSE is 1.8182110736
For rank 12 the RMSE is 1.34319667706
The best model was trained with rank 12


In [74]:
"""
Spark will soon deprecate MLLIb package. 
They are focusing more on ML packages with standard machine learning implementation
Let's see that package also
"""
als =  mlals(maxIter=iterations,rank=4,seed=seed,regParam=regularization_parameter, userCol="userId", itemCol="movieId",ratingCol="rating")
modelML = als.fit(trainingData)
pred = modelML.transform(validationData)
pred = pred.where(pred['prediction'] != 'NaN')
    
# Evaluate the model by computing RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(pred)

print 'RMSE is %s' % rmse

"""
The best part is we do not have to worry about RDD any more with this library
"""

RMSE is 1.94327935771


'\nThe best part is we do not have to worry about RDD any more with this library\n'

In [75]:
# Let's take test dataset and get ratings
predictions_test = model.predictAll(test_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))

In [76]:
## visualize preditions, here third element is predictions generated by ALS Model
predictions_test.take(3)

[((200, 3), 3.5267400298734084),
 ((460, 7), 3.2325654969628546),
 ((240, 16), 3.8609647883036122)]

In [77]:
"""
Let's start recommending movies.
I have written a method to call recommendations for a perticular user from test data

TODO: You need to execute one more step before calling getRecommendations, 
      Think about that step. If you go through the seps below, you will realize it soon.
"""
def getRecommendations(user,testDf,trainDf,model):
    # get all user and his/her rated movies
    userDf = testDf.filter(testDf.userId == user)
    # filter movies from main set which have not been rated by selected user
    # and pass it to model we sreated above
    mov = trainDf.select('movieId').subtract(userDf.select('movieId'))
    
    # Again we need to covert our dataframe into RDD
    pred_rat = model.predictAll(mov.rdd.map(lambda x: (user, x[0]))).collect()
    
    # Get the top recommendations
    recommendations = sorted(pred_rat, key=lambda x: x[2], reverse=True)[:50]
    
    return recommendations

In [78]:
# Assign user id for which we need recommendations
user = 269

# Call getRecommendations method
derived_rec = getRecommendations(user,testData,trainingData,model)

print ("Movies recommended for:%d" % user)



Movies recommended for:269


In [79]:
derived_rec


[Rating(user=269, product=11, rating=4.986591823580024),
 Rating(user=269, product=5, rating=4.821455620610955),
 Rating(user=269, product=1, rating=4.566817616998074),
 Rating(user=269, product=12, rating=4.413941354854723),
 Rating(user=269, product=8, rating=4.3757624175676),
 Rating(user=269, product=7, rating=4.244668235527135),
 Rating(user=269, product=14, rating=4.036707005084119),
 Rating(user=269, product=20, rating=4.001234231546155),
 Rating(user=269, product=21, rating=4.000386830223012),
 Rating(user=269, product=9, rating=3.851294238452813),
 Rating(user=269, product=13, rating=3.7756791741228763),
 Rating(user=269, product=18, rating=3.7467457425398494),
 Rating(user=269, product=6, rating=3.478718947169999),
 Rating(user=269, product=10, rating=3.4291735302826583),
 Rating(user=269, product=19, rating=3.204040969202517),
 Rating(user=269, product=15, rating=3.03762330633538),
 Rating(user=269, product=17, rating=3.0011408345368658)]

In [62]:
# Print the result
# TODO: we can convert derived_rec into a dataframe to present it properly
for i in xrange(len(derived_rec)):
    print i+1
    movies_df.filter(movies_df.movieId==derived_rec[i][1]).select('title').show()

1
+-----------------+
|            title|
+-----------------+
|Lord of the Rings|
+-----------------+

2
+-------------+
|        title|
+-------------+
|Despicable Me|
+-------------+

3
+------------+
|       title|
+------------+
|Pretty woman|
+------------+

4
+---------+
|    title|
+---------+
|Godfather|
+---------+

5
+------+
| title|
+------+
|Trolls|
+------+

6
+---------+
|    title|
+---------+
|Lala Land|
+---------+

7
+------------------+
|             title|
+------------------+
|500 days of Summer|
+------------------+

8
+--------+
|   title|
+--------+
|Hangover|
+--------+

9
+---------------+
|          title|
+---------------+
|Captain America|
+---------------+

10
+-----+
|title|
+-----+
| Kubo|
+-----+

11
+------------+
|       title|
+------------+
|Ghostbusters|
+------------+

12
+------------+
|       title|
+------------+
|Pulp Fiction|
+------------+

13
+-------------+
|        title|
+-------------+
|Almost Famous|
+-------------+

14
+-------------